In [ ]:
%matplotlib inline

import os
import sys
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
def read_H1data_from_file(file_name):
    """
    Reads csv files containing H1-like information (entries and bins),
    generated by the Geant4 Analysis Manager.
    """
    entries = []
    with open(file_name, 'r') as f:
    
        buff = f.readlines()
        for line in buff:
            if line[0] == '#':
                chunks = line.split(" ")
                if chunks[0] != "#axis":
                    continue
                else:
                    binl       = line.split(" ")
                    n_bins     = int(binl[2].split("\n")[0])
                    first_edge = float(binl[3].split("\n")[0])
                    last_edge  = float(binl[4].split("\n")[0])
                
                    continue
    
            l = line.split(",")
            if l[0] == 'entries':
                continue
        
            entries.append(int(l[0]))
            
    return entries, first_edge, last_edge, n_bins
    

In [ ]:
def create_H1data_from_csv(file_name):
    """
    Reads the entries of a H1 histogram and transforms them to individual data,
    replicating each bin for its number of entries. 
    The underflow and overflow bins are removed.
    """
    
    entries, first_edge, last_edge, n_bins = read_H1data_from_file(file_name)

    edges      = np.linspace(first_edge, last_edge, n_bins+1)
    left_edges = edges[:-1] # remove last right edge
    bin_width  = left_edges[1] - left_edges[0]

    entries = entries[1:-1] # remove underflow and overflow bins
    
    data = np.repeat(left_edges, entries, axis=0)
    
    return data, first_edge, last_edge, bin_width

In [ ]:
def create_H2data_from_csv(file_name):
    """
    Reads the entries of a H1 histogram and transforms them to individual data,
    replicating each bin for its number of entries. 
    The underflow and overflow bins are kept, to keep the total number of entries.
    This is needed to combine two sets of data in a 2D histogram, 
    which needs the same number of data in both sets.
    """

    entries, first_edge, last_edge, n_bins = read_H1data_from_file(file_name)

    edges      = np.linspace(first_edge, last_edge, n_bins+1)
    bin_width  = edges[1] - edges[0]
    ovf_edge = edges[0]-bin_width
    edges = np.insert(edges, 0, ovf_edge)
    
    data = np.repeat(edges, entries, axis=0)
    
    return data, first_edge, last_edge, bin_width

In [ ]:
path = '/path/to/folder/'

In [ ]:
# Number of scintillation photons produced per event

fname    = 'OpticalEvtInfo.csv'
tot_name = path + fname

photons = []
with open(tot_name, 'r') as f:
    
    buff = f.readlines()
    for line in buff:
        if line[0] == '#':
            continue
        #print(line)
        ph = float(line.split("\n")[0])
        photons.append(ph)

In [ ]:
plt.hist(photons, bins=10);

In [ ]:
# Wavelength of Cherenkov photons (nm)

fname    = 'OptTrackInfo_h1_CherLambda.csv'
tot_name = path + fname

data, _, _, _ = create_H1data_from_csv(tot_name)

plt.hist(data, bins=50, range=(0, 1500));

In [ ]:
# Wavelength of scintillation photons (nm)

fname    = 'OptTrackInfo_h1_ScintLambda.csv'
tot_name = path + fname

data, _, _, _ = create_H1data_from_csv(tot_name)

plt.hist(data, bins=30, range=(150, 200));

In [ ]:
# Wavelength of detected photons (nm)

fname    = 'OptTrackInfo_h1_PhLambdaDet.csv'
tot_name = path + fname

data, _, _, _ = create_H1data_from_csv(tot_name)

plt.hist(data, bins=50, range=(150, 200));

In [ ]:
# Velocity of scintillation photons (mm/ps)

fname    = 'OptTrackInfo_h1_PhVelocity.csv'
tot_name = path + fname

data, _, _, _ = create_H1data_from_csv(tot_name)

plt.hist(data, bins=50, range=(0, 0.4));

In [ ]:
# Scintillation time (ps)

fname    = 'OptTrackInfo_h1_ScintillationTime.csv'
tot_name = path + fname

data, _, _, _ = create_H1data_from_csv(tot_name)

plt.hist(data, bins=50);

In [ ]:
# 2D histogram of scintillation photon wavelength (nm) vs velocity (mm/ps)

fname    = 'OptTrackInfo_h1_PhLambdaDet.csv'
tot_name = path + fname

wvl, f1, l1, w1 = create_H2data_from_csv(tot_name)

fname    = 'OptTrackInfo_h1_PhVelocity.csv'
tot_name = path + fname

vel, f2, l2, w2 = create_H2data_from_csv(tot_name)

In [ ]:
plt.hist2d(vel, wvl, bins=(50, 50), range=((0.1, 0.3), (150, 200)), cmin=1);

plt.colorbar();